# 📈 Whylogs Profile Visualization
### gives you various ways to simplify the proccess of detecting dataset drift. Bellow you can see the instructions for creating dummy dataset and list of currently available visual reports.

### 🗂️Install dependencies and make imports

In [ ]:
!pip install faker

In [ ]:
import numpy as np
import datetime
from collections import OrderedDict
from faker import Faker

In [ ]:
from whylogs import get_or_create_session
from whylogs.viz import NotebookProfileViewer

### ♻️Create dummy data

In [ ]:
locales = OrderedDict([
    ('en-US', 1),
    ('fr-FR', 2),
    ('ja_JP', 2),
])
fake = Faker(locales)
distribution = np.concatenate((np.random.normal(0.1, 0.1, 500), np.random.normal(0.6, 0.2, 500)))

### 📝 Log it


In [ ]:
session = get_or_create_session()
def profile_generator():
    with session.logger("mytestytest", dataset_timestamp=datetime.datetime(2021, 6, 2)) as logger:
        for _ in range(500):
            logger.log({"uniform_integers": np.random.randint(0,50)})
            logger.log({"strings": fake.name()})
            logger.log({"mixture_distribution": np.random.choice(distribution, 1)[0]})
            logger.log({"nulls": None})
        logger.log({"moah_data": 1})
        logger.log({"moah_data": 1})
        logger.log({"moah_data": 5})
        return logger.profile

target_profile = profile_generator()
reference_profile = profile_generator() 

## ✨ Vizualize profiles with Whylogs

### Initialization
Initialize Profile viewer by passing profiles for which you want to get the visualizations

In [ ]:
# should we allow multiple profiles?
visualization = NotebookProfileViewer(target_profiles=[target_profile], reference_profiles=[reference_profile])

###### `*target_profiles`: Profiled dataset which will be reffered as `target`
###### `*reference_profiles`: Profiled dataset which will be reffered as `reference`

### Summary Drift Report

You can get summary drift report for `target` and `reference` profiles features

In [ ]:
visualization.summary_drift_report(preferred_cell_height="1000px")

###### `preferred_cell_height`: height in `px` for generated visualization cell 

### Double histogram

You can get double histogram for any of your features

In [ ]:
visualization.double_histogram(feature_names="uniform_integers")

###### `*feature_names`: string or list of strings containing names of the features for which you want to see double histogram
###### `preferred_cell_height`: height in `px` for generated visualization cell 

### Summary Statistics

You can get set of useful statistics for both `target` and `reference` by passing the name

In [ ]:
visualization.summary_statistics(profile="target")

###### `profile_name`: `"target"` or `"reference"`
###### `prefered_cell_height`: height in `px` for generated visualization cell 

### Feature Summary Statistics

You can get set of useful statistics for features by passing the profile and feature names

In [ ]:
visualization.feature_summary_statistics(feature_name"uniform_integers", profile="reference")

###### `*feature_name`: Any feature name from your profiled dataset
###### `profile_name`: `"target"` or `"reference"`
###### `prefered_cell_height`: height in `px` for generated visualization cell 

### Download prefered cell output

You can also download any of those visualisation in `HTML` format for further analysys, by passing the visualization name

In [ ]:
visualization.download(html=visualization.summary_drift_report(), html_file_name='example')

By calling `download()` method of `DisplayProfile` and passing visualizer command, path to be downloaded to (optional) and name of the file you prefer (optional). Exaple `download(visualization.feature("title"), path="examlpe/path", html_file_name="example_html_file_name")`. Command will download HTML format.

If path is not passed file will be downloaded to `html_reports` located in whylogs directory by default.

If name of the file is not passed it will be name of the dataset followed by timestamp of the profile by default. 

###### `*feature_name`: Any feature name from your profiled dataset
###### `preferred_path`: save path `default:` `/html_reports` located in whylogs directory
###### `html_file_name`: name of the file `default:` name of the dataset followed by timestamp of the profile